In [2]:
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [68]:
from tqdm.auto import tqdm

In [67]:
import re

In [3]:
data = pd.read_csv("./uiks-utf8.csv")

In [4]:
data.head()

,region_name,tik_name,uik_name,registered_voters,used_ballots,found_ballots,incorrect_ballots,yes_votes,no_votes
0,Республика Адыгея (Адыгея),1 Адыгейская,УИК №1,2277,1925,1925,0,1749,176
1,Республика Адыгея (Адыгея),1 Адыгейская,УИК №2,2694,2290,2290,7,2117,166
2,Республика Адыгея (Адыгея),1 Адыгейская,УИК №3,2865,2441,2441,11,2171,259
3,Республика Адыгея (Адыгея),1 Адыгейская,УИК №4,2054,1669,1669,8,1381,280
4,Республика Адыгея (Адыгея),1 Адыгейская,УИК №5,714,607,607,1,473,133


In [66]:
locations = pd.read_csv("cik_uik.csv")

In [71]:
locations = locations[locations.type_ik == 'uik'].fillna("")

In [72]:
lat = []
lon = []
miss = 0
hits = 0

for x in tqdm(data.itertuples(), total=len(data)):
    normal_region = re.sub("\(.+?\)", "", x.region_name) ## removing (smth) from region_name to make it compatible with cik_uik addresses
    normal_uik    = x.uik_name.replace("УИК", "Участковая избирательная комиссия") ## exapanding uik abbrev.
    retrieval     = locations[locations.address.str.contains(normal_region)]
    retrieval     = retrieval[retrieval.name.str.contains(normal_uik)]
    if len(retrieval) != 1:
        miss += 1
        lat.append(None)
        lon.append(None)
    else:
        hits += 1
        lat.append(retrieval.iloc[0]['lat_voteroom'])
        lon.append(retrieval.iloc[0]['lon_voteroom'])

In [73]:
print(f"{(hits/len(data)) * 100:.4f}% locations found")

80.4654% locations found


In [74]:
data['latitude'] = lat
data['longitude'] = lon

In [78]:
data['longitude'] = data['longitude'].apply(lambda x: float(x) if x else float("NaN"))

In [79]:
data['latitude'] = data['latitude'].apply(lambda x: float(x) if x else float("NaN"))

In [85]:
data.to_csv("uiks-sig.csv", index=False, encoding='utf-8-sig')
data.to_csv("uiks-utf8.csv", index=False, encoding='utf-8')